In [70]:
:dep plotters = { version = "^0.3.5", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
:dep showata = { version = "0.3", features=["show_ndarray"]}
:dep ndarray = "0.14"
:dep petgraph = "0.5.1"
:dep petgraph-evcxr = "0.2.0"
:dep uuid = "1.8.0"

extern crate plotters;
use plotters::prelude::*;
use showata::Showable;
use ndarray::{
    prelude::*,
    arr1,
    Array,
    Array2
};
use std::collections::HashMap;
use std::{cell::RefCell, fmt, ops, rc::Rc};

// use petgraph::dot::Dot;
use petgraph::prelude::{DiGraph, NodeIndex};
use petgraph::dot::{Config, Dot};
use petgraph_evcxr::{draw_graph, draw_graph_with_attr_getters, draw_dot};
use uuid::Uuid;

In [71]:
#[derive(Debug)]
pub enum Op {
    Add,
    Mul,
}

#[derive(fmt::Debug)]
pub struct ValueInfo {
    pub id: Uuid,
    pub label: String,
    pub value: f64,
    pub prev: Vec<Value>,
    pub op: Option<Op>,
}

#[derive(Clone)]
pub struct Value(Rc<RefCell<ValueInfo>>);

impl PartialEq for Value {
    fn eq(&self, other: &Self) -> bool {
        self.0.borrow().id == other.0.borrow().id
    }
}

impl Eq for Value {}

impl ops::Deref for Value {
    type Target = Rc<RefCell<ValueInfo>>;

    fn deref(&self) -> &Self::Target {
        &self.0
    }
}

impl fmt::Debug for Value {
    fn fmt(&self, f: &mut fmt::Formatter<'_>) -> std::fmt::Result {
        write!(f, "Value: {}", self.0.borrow())
    }
}

impl fmt::Display for ValueInfo {
    fn fmt(&self, f: &mut fmt::Formatter<'_>) -> std::fmt::Result {
        write!(
            f,
            "(value: {}, prev: {:?}, op: {:?})",
            self.value, self.prev, self.op
        )
    }
}

impl Value {
    pub fn new(value: f64, label: &str) -> Value {
        Value(Rc::new(RefCell::new(ValueInfo {
            id: Uuid::default(),
            label: label.to_string(),
            value,
            prev: Vec::new(),
            op: None,
        })))
    }

    pub fn mul(&self, other: &Value, label: &str) -> Value {
        let new = Value(Rc::new(RefCell::new(ValueInfo {
            id: Uuid::default(),
            label: label.to_string(),
            value: self.0.borrow().value * other.0.borrow().value,
            prev: vec![self.clone(), other.clone()],
            op: Some(Op::Mul),
        })));
        new
    }

    pub fn add(&self, other: &Value, label: &str) -> Value {
        let new = Value(Rc::new(RefCell::new(ValueInfo {
            id: Uuid::default(),
            label: label.to_string(),
            value: self.0.borrow().value + other.0.borrow().value,
            prev: vec![self.clone(), other.clone()],
            op: Some(Op::Add),
        })));
        new
    }
}

In [76]:
type Graph = DiGraph<String, String>;
type NodeHashMap = HashMap<Uuid, NodeIndex>;

fn recursive_build(graph: &mut Graph, node_hash_map: &mut NodeHashMap, node: &Value) -> NodeIndex {
    if let Some(&node_index) = node_hash_map.get(&node.borrow().id) {
        return node_index;
    }

    let node_index = graph.add_node(format!("data: {:.2}", node.borrow().value));

    node_hash_map.insert(node.borrow().id, node_index);

    for child in node.borrow().prev.iter() {
        let child_index = recursive_build(graph, node_hash_map, child);
        graph.add_edge(
            child_index,
            node_index,
            format!("op: {:?}", node.borrow().op),
        );
    }
    node_index
}

fn build_graph(root_node: &Value) -> Graph {
    let mut graph = DiGraph::<String, String>::new();
    let mut node_hash_map = HashMap::<Uuid, NodeIndex>::new();

    recursive_build(&mut graph, &mut node_hash_map, root_node);

    graph
}

pub fn create_graphviz(root_node: &Value) -> Graph {
    let graph = build_graph(root_node);

    let dot = petgraph::dot::Dot::with_config(&graph, &[Config::EdgeNoLabel]);

    graph
}



In [77]:
let a = Value::new(2.0, "a");
let b = Value::new(-3.0, "b");
let c = Value::new(10.0, "c");
let d = a.mul(&b, "d");
let e = d.add(&c, "e");
e

Value: (value: 4, prev: [Value: (value: -6, prev: [Value: (value: 2, prev: [], op: None), Value: (value: -3, prev: [], op: None)], op: Some(Mul)), Value: (value: 10, prev: [], op: None)], op: Some(Add))

In [80]:
let graph = create_graphviz(&e);
draw_graph(&graph);

Error: the trait bound `petgraph::Graph<String, String>: petgraph::visit::Data` is not satisfied

Error: the trait bound `petgraph::Graph<String, String>: petgraph::visit::NodeIndexable` is not satisfied

Error: the trait bound `petgraph::Graph<String, String>: petgraph::visit::IntoNodeReferences` is not satisfied

Error: the trait bound `petgraph::Graph<String, String>: petgraph::visit::IntoEdgeReferences` is not satisfied

Error: the trait bound `petgraph::Graph<String, String>: petgraph::visit::GraphProp` is not satisfied